<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#A3:-A*,-IDS,-and-Effective-Branching-Factor" data-toc-modified-id="A3:-A*,-IDS,-and-Effective-Branching-Factor-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>A3: A*, IDS, and Effective Branching Factor</a></span><ul class="toc-item"><li><span><a href="#Heuristic-Functions" data-toc-modified-id="Heuristic-Functions-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Heuristic Functions</a></span></li><li><span><a href="#Comparison" data-toc-modified-id="Comparison-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Comparison</a></span></li><li><span><a href="#Grading" data-toc-modified-id="Grading-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Grading</a></span></li><li><span><a href="#Extra-Credit" data-toc-modified-id="Extra-Credit-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Extra Credit</a></span></li></ul></li></ul></div>

# A3: A\*, IDS, and Effective Branching Factor

For this assignment, implement the Recursive Best-First Search
implementation of the A\* algorithm given in class.  Name this function `Astar_search`.  Also in this notebook include your `iterative_deepening_search` functions.
Define a new function named `effective_branching_factor` that returns an estimate of the effective
branching factor for a search algorithm applied to a search problem.

So, the required functions are

   - `Astar_search(start_state, actions_f, take_action_f, goal_test_f, h_f)`
   - `iterative_deepening_search(start_state, goal_state, actions_f, take_action_f, max_depth)`
   - `effective_branching_factor(n_nodes, depth, precision=0.01)`, returns the effective branching factor, given the number of nodes expanded and depth reached during a search.

Apply `iterative_deepening_search` and `Astar_search` to several eight-tile sliding puzzle
problems. For this you must include your implementations of these functions from Assignment 2. Here we are renaming these functions to not include `_f`, just for simplicity.

  * `actions_8p(state)`: returns a list of up to four valid actions that can be applied in `state`. With each action include a step cost of 1. For example, if all four actions are possible from this state, return [('left', 1), ('right', 1), ('up', 1), ('down', 1)].
  * `take_action_8p(state, action)`: return the state that results from applying `action` in `state` and the cost of the one step,
  
plus the following function for the eight-tile puzzle:

  * `goal_test_8p(state, goal)`
  
Compare their results by displaying
solution path depth, number of nodes 
generated, and the effective branching factor, and discuss the results.  Do this by defining the following function that prints the table as shown in the example below.

   - `run_experiment(goal_state_1, goal_state_2, goal_state_3, [h1, h2, h3])`
   
Define this function so it takes any number of $h$ functions in the list that is the fourth argument.

In [1]:
#global variable for n_nodes_expanded
global n_nodes_expanded
n_nodes_expanded = 0

In [2]:
#adapted from lecture notes 07
# Recursive Best First Search (Figure 3.26, Russell and Norvig)
#  Recursive Iterative Deepening form of A*, where depth is replaced by f(n)
class Node:

    def __init__(self, state, f=0, g=0, h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h

    def __repr__(self):
        return f'Node({self.state}, f={self.f}, g={self.g}, h={self.h})'

def Astar_search(start_state, actions_f, take_action_f, goal_test_f, h_f):
    #CHANGED IN A3: added global variable to keep track of number of nodes expanded
    global n_nodes_expanded
    n_nodes_expanded = 0
    h = h_f(start_state)
    start_node = Node(state=start_state, f=0 + h, g=0, h=h)
    return a_star_search_helper(start_node, actions_f, take_action_f, 
                                goal_test_f, h_f, float('inf'))

def a_star_search_helper(parent_node, actions_f, take_action_f, 
                         goal_test_f, h_f, f_max):
    global n_nodes_expanded
    if goal_test_f(parent_node.state):
        return ([parent_node.state], parent_node.g)
    
    ## Construct list of children nodes with f, g, and h values
    actions = actions_f(parent_node.state)
    if not actions:
        return ('failure', float('inf'))
    
    children = []
    for action in actions:
        #CHANGED: increment expanded nodes by 1
        n_nodes_expanded += 1
        (child_state, step_cost) = take_action_f(parent_node.state, action)
        h = h_f(child_state)
        g = parent_node.g + step_cost
        f = max(h + g, parent_node.f)
        child_node = Node(state=child_state, f=f, g=g, h=h)
        children.append(child_node)
        
    while True:
        # find best child
        children.sort(key = lambda n: n.f) # sort by f value
        best_child = children[0]
        if best_child.f > f_max:
            return ('failure', best_child.f)
        # next lowest f value
        alternative_f = children[1].f if len(children) > 1 else float('inf')
        # expand best child, reassign its f value to be returned value
        result, best_child.f = a_star_search_helper(best_child, actions_f,
                                                    take_action_f, goal_test_f,
                                                    h_f,
                                                    min(f_max,alternative_f))
        if result != 'failure':                    #        g
            result.insert(0, parent_node.state)    #       / 
            return (result, best_child.f)          #      d
                                                   #     / \ 
#if __name__ == "__main__":                         #    b   h   
#                                                   #   / \   
#    successors = {'a': ['b','c'],                  #  a   e  
#                  'b': ['d','e'],                  #   \         
#                  'c': ['f'],                      #    c   i
#                  'd': ['g', 'h'],                 #     \ / 
#                  'f': ['i','j']}                  #      f  
#                                                   #      \
#    def actions_f(state):                          #       j 
#        try:
#            ## step cost of each action is 1
#            return [(succ, 1) for succ in successors[state]]
#        except KeyError:
#            return []
#
#    def take_action_f(state, action):
#        return action
#
#    def goal_test_f(state):
#        return state == goal
#
#    def h1(state):
#        return 0
#
#    start = 'a'
#    goal = 'h'
#    result = a_star_search(start, actions_f, take_action_f, goal_test_f, h1)
#
#    print(f'Path from a to h is {result[0]} for a cost of {result[1]}')
#    return "No A* search yet"

In [3]:
#from A2
def iterative_deepening_search(start_state, goal_state, actions_f, take_action_f, max_depth):
    
    #CHANGED IN A3: added global variable to keep track of number of nodes expanded
    global n_nodes_expanded
    n_nodes_expanded = 0
    
    # Conduct multiple searches, starting with smallest depth, then increasing it by 1 each time.
    for depth in range(max_depth):
        
        # Conduct search from startState
        result = depth_limited_search(start_state, goal_state, actions_f, take_action_f, depth)
        
        # If result was failure, return 'failure'.
        if result is 'failure':
            return 'failure'
        
        # Otherwise, if result was not cutoff, it succeeded, so add start_state to solution path and return it.
        if result is not 'cutoff':
            #Add start_state to front of solution path, in result, returned by depth_limited_search   
            result.insert(0, start_state)
            
            return result
        
    # If we reach here, no solution found within the max_depth limit.
    return 'cutoff'

In [4]:
#adapted from A2
def depth_limited_search(state, goal_state, actions_f, take_action_f, depth_limit):
    
    #CHANGED IN A3: added global variable to keep track of number of nodes expanded
    global n_nodes_expanded
    # If we have reached the goal, exit, returning an empty solution path.
    #If state == goal_state, then
    if state == goal_state:
        return []
    
    # If we have reached the depth limit, return the string 'cutoff'.
    if depth_limit is 0:
        #Return the string 'cutoff' to signal that the depth limit was reached
        return 'cutoff'
        
    cutoff_occurred = False
    
    # For each possible action from state ...
    for action in actions_f(state):
        #CHANGED IN A3: increment nodes expanded counter
        n_nodes_expanded+= 1
        # Apply the action to the current state to get a next state, named child_state
        #CHANGED IN A3: only use the state, not the step cost
        child_state = take_action_f(state, action)[0]
        
        # Recursively call this function to continue the search starting from the child_state.
        # Decrease by one the depth_limit for this search.
        result = depth_limited_search(child_state, goal_state, actions_f, take_action_f, depth_limit - 1)
        
        # If result was 'cufoff', just note that this happened.
        if result is 'cutoff':
            cutoff_occurred = True
            
        # If result was not 'failure', search succeeded so add childState to front of solution path and
        # return that path.
        elif result is not 'failure':
            #Add child_state to front of partial solution path, in result, returned by depth_limited_search
            result.insert(0, child_state)
            return result
        
    # We reach here only if cutoff or failure occurred.  Return whichever occurred.
    if cutoff_occurred:
        return 'cutoff'
    else:
        return 'failure'

In [5]:
#adapted from lecture notes 10
def effective_branching_factor(n_nodes, depth, precision=0.01):
    lower = 1
    upper = n_nodes
    b_est = 0
    n_est = 0
    while n_est > n_nodes+precision or n_est < n_nodes-precision:
        print(lower, upper)
        b_est = (lower+upper)/2
        #to prevent division by 0
        if lower == upper:
            return upper
        else:
            n_est = (1-b_est**(depth+1))/(1-b_est)
        if n_est > n_nodes:
            upper = b_est
        if n_est < n_nodes:
            lower = b_est
    return b_est

In [6]:
#modified from A2
def find_blank_8p(state):
    #from lecture 9/8/2020
    #if step cost is included then just look at state
    if len(state) == 2:
        zero_at = state[0].index(0)
    else:
        zero_at = state.index(0)
    return zero_at // 3, zero_at % 3

In [7]:
#from A2
def print_state_8p(state):
    print_state = ['-' if x == 0 else x for x in state]
    print_string = ''
    for i in range(len(print_state)):
        if print_state[i] == '-':
            print_string += '- '
        else: 
            print_string += str(print_state[i])
            print_string += ' '
        if i % 3 == 2:
            print_string += "\n"
    print(print_string)

In [8]:
#modified from A2
def actions_8p(state):
    actions = []
    s_cost = 1
    if len(state) == 2:
        s_cost = state[1]
    blank = find_blank_8p(state)
    if blank[1] != 0:
        actions.append(('left', s_cost))
    if blank[1] != 2:
        actions.append(('right', s_cost))
    if blank[0] != 0:
        actions.append(('up', s_cost))
    if blank[0] != 2:
        actions.append(('down', s_cost))
    return actions

In [9]:
#modified from A2
def take_action_8p(state, action):
    s_cost = 1
    if len(state) == 2:
        s_cost = state[1]
        return_state = state[0].copy()
    else:
        return_state = state.copy()
    actions = actions_8p(state)
    blank = find_blank_8p(state)
    if len(state) == 2:
        blank_index = state[0].index(0)
    else:
        blank_index = state.index(0)
    acopy = actions.copy()
    for t in range(len(acopy)):
        acopy[t] = acopy[t][0]
    if action[0] in acopy:
        if action[0] == 'left':
            return_state[blank_index] = return_state[blank_index-1]
            return_state[blank_index-1] = 0
        elif action[0] == 'right':
            return_state[blank_index] = return_state[blank_index+1]
            return_state[blank_index+1] = 0
        elif action[0] == 'up':
            return_state[blank_index] = return_state[blank_index-3]
            return_state[blank_index-3] = 0
        elif action[0] == 'down':
            return_state[blank_index] = return_state[blank_index+3]
            return_state[blank_index+3] = 0
    return (return_state, s_cost)

In [10]:
def goal_test_8p(state, goal):
    if state == goal:
        return True
    else:
        return False

## Heuristic Functions

For `Astar_search` use the following two heuristic functions, plus one more of your own design, for a total of three heuristic functions.

  * `h1_8p(state, goal)`: $h(state, goal) = 0$, for all states $state$ and all goal states $goal$,
  * `h2_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance that the blank is from its goal position,
  * `h3_8p(state, goal)`: $h(state, goal) = ?$, that you define.  It must be admissible, and not constant for all states.

In [11]:
def h1_8p(state, goal):
    return 0

In [12]:
def h2_8p(state, goal):
    state_blank = find_blank_8p(state)
    goal_blank = find_blank_8p(goal)
    r_dist = abs((state_blank[0]-goal_blank[0]))
    col_dist = abs((state_blank[1]-goal_blank[1]))
    m_dist = r_dist + col_dist
    return m_dist

In [13]:
#this will be a "bad" heuristic function, it will only account for horizontal/column distance between blank and goal
#therefore is never overestimates, it will only underestimate, since it does not take into account the vertical/row distance
#since it never overestimates it is admissible
def h3_8p(state, goal):
    state_blank = find_blank_8p(state)
    goal_blank = find_blank_8p(goal)
    col_dist = abs((state_blank[1]-goal_blank[1]))
    return col_dist

## Comparison

Apply all four algorithms (`iterative_deepening_search` plus `Astar_search` with the three heuristic
functions) to three eight-tile puzzle problems with start state

$$
\begin{array}{ccc}
1 & 2 & 3\\
4 & 0 & 5\\
6 & 7 & 8
\end{array}
$$

and these three goal states.

$$
\begin{array}{ccccccccccc}
1 & 2 & 3  & ~~~~ & 1 & 2 & 3  &  ~~~~ & 1 & 0 &  3\\
4 & 0 & 5  & & 4 & 5 & 8  & & 4 & 5 & 8\\
6 & 7 & 8 &  & 6 & 0 & 7  & & 2 & 6 & 7
\end{array}
$$

Print a well-formatted table like the following.  Try to match this
format. If you have time, you might consider learning a bit about the `DataFrame` class in the `pandas` package.  When displayed in jupyter notebooks, `pandas.DataFrame` objects are nicely formatted in html.

           [1, 2, 3, 4, 0, 5, 6, 7, 8]    [1, 2, 3, 4, 5, 8, 6, 0, 7]    [1, 0, 3, 4, 5, 8, 2, 6, 7] 
    Algorithm    Depth  Nodes  EBF              Depth  Nodes  EBF              Depth  Nodes  EBF          
         IDS       0      0  0.000                3     43  3.086               11 225850  2.954         
        A*h1       0      0  0.000                3    116  4.488               11 643246  3.263         
        A*h2       0      0  0.000                3     51  3.297               11 100046  2.733         

Of course you will have one more line for `h3`.

First, some example output for the effective_branching_factor function.  During execution, this example shows debugging output which is the low and high values passed into a recursive helper function.

In [14]:
effective_branching_factor(10, 3)

1 10
1 5.5
1 3.25
1 2.125
1.5625 2.125
1.5625 1.84375
1.5625 1.703125
1.6328125 1.703125
1.6328125 1.66796875
1.650390625 1.66796875
1.6591796875 1.66796875
1.6591796875 1.66357421875


1.661376953125

The smallest argument values should be a depth of 0, and 1 node.

In [15]:
effective_branching_factor(1, 0)

1 1


1

In [16]:
effective_branching_factor(2, 1)

1 2
1 1.5
1 1.25
1 1.125
1 1.0625
1 1.03125
1 1.015625


1.0078125

In [17]:
effective_branching_factor(2, 1, precision=0.000001)

1 2
1 1.5
1 1.25
1 1.125
1 1.0625
1 1.03125
1 1.015625
1 1.0078125
1 1.00390625
1 1.001953125
1 1.0009765625
1 1.00048828125
1 1.000244140625
1 1.0001220703125
1 1.00006103515625
1 1.000030517578125
1 1.0000152587890625
1 1.0000076293945312
1 1.0000038146972656
1 1.0000019073486328


1.0000009536743164

In [18]:
effective_branching_factor(200000, 5)

1 200000
1 100000.5
1 50000.75
1 25000.875
1 12500.9375
1 6250.96875
1 3125.984375
1 1563.4921875
1 782.24609375
1 391.623046875
1 196.3115234375
1 98.65576171875
1 49.827880859375
1 25.4139404296875
1 13.20697021484375
7.103485107421875 13.20697021484375
10.155227661132812 13.20697021484375
10.155227661132812 11.681098937988281
10.918163299560547 11.681098937988281
10.918163299560547 11.299631118774414
11.10889720916748 11.299631118774414
11.204264163970947 11.299631118774414
11.25194764137268 11.299631118774414
11.25194764137268 11.275789380073547
11.263868510723114 11.275789380073547
11.26982894539833 11.275789380073547
11.272809162735939 11.275789380073547
11.274299271404743 11.275789380073547
11.275044325739145 11.275789380073547
11.275416852906346 11.275789380073547
11.275416852906346 11.275603116489947
11.275509984698147 11.275603116489947
11.275556550594047 11.275603116489947
11.275579833541997 11.275603116489947
11.275591475015972 11.275603116489947
11.275591475015972 11.27559

11.275596931956898

In [19]:
effective_branching_factor(200000, 50)

1 200000
1 100000.5
1 50000.75
1 25000.875
1 12500.9375
1 6250.96875
1 3125.984375
1 1563.4921875
1 782.24609375
1 391.623046875
1 196.3115234375
1 98.65576171875
1 49.827880859375
1 25.4139404296875
1 13.20697021484375
1 7.103485107421875
1 4.0517425537109375
1 2.5258712768554688
1 1.7629356384277344
1 1.3814678192138672
1.1907339096069336 1.3814678192138672
1.1907339096069336 1.2861008644104004
1.1907339096069336 1.238417387008667
1.2145756483078003 1.238417387008667
1.2264965176582336 1.238417387008667
1.2324569523334503 1.238417387008667
1.2324569523334503 1.2354371696710587
1.2339470610022545 1.2354371696710587
1.2346921153366566 1.2354371696710587
1.2346921153366566 1.2350646425038576
1.2346921153366566 1.234878378920257
1.2347852471284568 1.234878378920257
1.2347852471284568 1.234831813024357
1.234808530076407 1.234831813024357
1.234808530076407 1.234820171550382
1.2348143508133944 1.234820171550382
1.2348172611818882 1.234820171550382
1.234818716366135 1.234820171550382
1.23481

1.2348192492705223

Here is a simple example using our usual simple graph search.

In [20]:
def actions_simple(state):
    succs = {'a': ['b', 'c'], 'b':['a'], 'c':['h'], 'h':['i'], 'i':['j', 'k', 'l'], 'k':['z']}
    return [(s, 1) for s in succs.get(state, [])]

def take_action_simple(state, action):
    return action

def goal_test_simple(state, goal):
    return state == goal

def h_simple(state, goal):
    return 1

In [21]:
actions = actions_simple('a')
actions

[('b', 1), ('c', 1)]

In [22]:
take_action_simple('a', actions[0])

('b', 1)

In [23]:
goal_test_simple('a', 'a')

True

In [24]:
h_simple('a', 'z')

1

In [25]:
iterative_deepening_search('a', 'z', actions_simple, take_action_simple, 10)

['a', 'c', 'h', 'i', 'k', 'z']

In [26]:
Astar_search('a',actions_simple, take_action_simple,
            lambda s: goal_test_simple(s, 'z'),
            lambda s: h_simple(s, 'z'))

(['a', 'c', 'h', 'i', 'k', 'z'], 5)

In [27]:
import pandas
def run_experiment(goal_state_1, goal_state_2, goal_state_3, h_list):
    global n_nodes_expanded
    goal_states = [goal_state_1, goal_state_2, goal_state_3]
    results = []
    sub_results = []
    sub_results.append("IDS")
    #run IDS for each goal state
    for goal_s in goal_states:
        IDS_results = iterative_deepening_search(state, goal_s, actions_8p, take_action_8p, 20)
        solution_depth = len(IDS_results)-1
        sub_results.append(solution_depth)
        sub_results.append(n_nodes_expanded)
        ebf = round(effective_branching_factor(n_nodes_expanded, solution_depth), 3)
        sub_results.append(ebf)
    results.append(sub_results)
    sub_results = []
    #run A* for each goal state for each heuristic function
    for h in h_list:
        #will not give correct heuristic name if it is defined differently from h1, h2, h3
        sub_results.append("A*"+str(h)[10:12])
        for goal_s in goal_states:
            Astar_results = Astar_search(state, actions_8p, take_action_8p, lambda s: goal_test_8p(s, goal_s), lambda s: h(s, goal_s))
            solution_depth = Astar_results[len(Astar_results)-1]
            sub_results.append(solution_depth)
            sub_results.append(n_nodes_expanded)
            ebf = round(effective_branching_factor(n_nodes_expanded, solution_depth), 3)
            sub_results.append(ebf)
        results.append(sub_results)
        sub_results = []
    results.append(sub_results)
    #get rid of excess result added
    results.pop(-1)
    print("results:",results)
    print("  [1, 2, 3, 4, 0, 5, 6, 7, 8][1, 2, 3, 4, 5, 8, 6, 0, 7][1, 0, 3, 4, 5, 8, 2, 6, 7] ")
    df = pandas.DataFrame(results, columns=('Algorithm', 'Depth', 'Nodes', 'EBF', '  Depth', 'Nodes', 'EBF', '  Depth', 'Nodes', 'EBF',))
    pandas.options.display.max_colwidth=10
    print(df)
    

In [28]:
state = [1, 2, 3, 4, 0, 5, 6, 7, 8]
goal_state_1 =  [1, 2, 3, 4, 0, 5, 6, 7, 8]
goal_state_2 =  [1, 2, 3, 4, 5, 8, 6, 0, 7]
goal_state_3 =  [1, 0, 3, 4, 5, 8, 2, 6, 7]
run_experiment(goal_state_1, goal_state_2, goal_state_3, [h1_8p, h2_8p, h3_8p])

1 43
1 22.0
1 11.5
1 6.25
1 3.625
2.3125 3.625
2.96875 3.625
2.96875 3.296875
2.96875 3.1328125
3.05078125 3.1328125
3.05078125 3.091796875
3.0712890625 3.091796875
3.08154296875 3.091796875
3.08154296875 3.086669921875
3.0841064453125 3.086669921875
3.08538818359375 3.086669921875
1 225850
1 112925.5
1 56463.25
1 28232.125
1 14116.5625
1 7058.78125
1 3529.890625
1 1765.4453125
1 883.22265625
1 442.111328125
1 221.5556640625
1 111.27783203125
1 56.138916015625
1 28.5694580078125
1 14.78472900390625
1 7.892364501953125
1 4.4461822509765625
2.7230911254882812 4.4461822509765625
2.7230911254882812 3.584636688232422
2.7230911254882812 3.1538639068603516
2.9384775161743164 3.1538639068603516
2.9384775161743164 3.046170711517334
2.9384775161743164 2.992324113845825
2.9384775161743164 2.965400815010071
2.9519391655921936 2.965400815010071
2.9519391655921936 2.958669990301132
2.9519391655921936 2.955304577946663
2.9536218717694283 2.955304577946663
2.9536218717694283 2.9544632248580456
2.95362

# Discussion Part 1: The third heuristic function

For my third heurstic function I implemented a "bad" heuristic function that outputs only the horizontal distance between the blank and the goal.  As such, it is worse than the second heuristic function, which accounts for both the horizontal and vertical distance to the goal.  I think that it is admissible since it never overestimates the distance to the goal.  In the case where the blank is on the same row as the goal, it will give the accurate distance from the goal.  In the case where the blank is on the same column as the goal it will give 0 distance from the goal.  And in the case where the blank is neither on the same row or same column as the goal it will give only the horizontal distance to the goal.  As such, in all these cases, the heursitic function will never overestimate the distance towards the goal: it will either give the correct distance or give an underestimate, since it does not account for vertical distance.  As such, since the heuristic function never overestimates the cost of the minimum cost path to the goal node, the heuristic function is admissible.  

# Discussion Part 2: Similiarities and differences of search results

The effective branching factor can be understood to be a measure of the number of nodes expanded to reach the goal state of a search, while also taking into account the depth that the solution was found at. As such, a higher effective branching factor means the search was less effective, in that it expanded more nodes before it reached the goal.  Therefore, effective branching factor can be used to measure the performance of a search.  For the first search problem, the start state was equal to the goal state, and as such all of the searches performed equally.  It can be seen that for the search problems chosen A* h1 performed the worst, with the highest effective branching factor for each (non-trivial) search problem attempted.  This is somewhat expected, since the h1 heuristic function always returns 0, giving no reliable indication of the distance from the blank to the goal.  IDS,  A* h2, and A* h3 performed better than A* h1 on all (non-trivial) occasions, having a lower effective branching factor on all search problems attempted.  A* h2 and A* h3 performed nearly evenly on the second search problem, but on the third search problem A* h2 produced a far lower effective branching factor than A* h3.  Seeing as the number of nodes expanded for the third search problem was much greater than for the second problem, I think that the more effective heuristic function of A* h2 was what allowed it to greatly outperform A* h3 for that problem.  For the second search problem, however, A* h3 actually had a slightly lower effective branching factor than  A* h2.  I think that this is because the solution for the 8 puzzle is more than simply moving the blank to the goal state.  As such, I think that even though the A* h3 heuristic function gave a less accurate indicator of the cost to move the blank to the goal state, it sometimes gave an answer that was more effective overall than the A* h2 heuristic function.  I think the difference in overall effectiveness between the A* h2 and A* h3 heuristic functions can still be seen clearly in the third search problem, which required many more moves to complete.  Since the third search problem required many more moves to complete, I think that the A* h2 heuristic function's higher accuracy had a greater overall effect than any randomness that may have produced a better result with A* h3.  I think the IDS search performed in a somewhat similar fashion to the A* h3 search, in that had a lower effective branching factor than A* h2 on the shorter second problem, but a greater effective branching factor on the third larger problem.  Since the IDS is an uninformed search, I think it performed better in the shorter search problem in the same way that A* h3 did, in that it had a higher chance of randomly running into the goal on a shorter problem.  On the longer search problem, however, I think the advantage of the informed A* h2 heuristic showed, as the A* h2 heuristic resulted in a lower effective branching factor than the IDS.  I think it is also worth noting that even on the longer search the IDS performed better than A* h3, showing that it takes a good heuristic function to make the A* search work effectively, which the A* h3 heuristic was not.  

## Grading

Download [A3grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A3grader.tar) and extract A3grader.py from it.

In [29]:
%run -i A3grader.py


======================= Code Execution =======================

Extracting python code from notebook named 'Valdes-A3.ipynb' and storing in notebookcode.py
Removing all statements that are not function or class defs or import statements.

Testing actions_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your actions_8p correctly returned [('left', 1), ('right', 1), ('up', 1)]

Testing take_action_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], (up, 1))

--- 5/5 points. Your take actions_8p correctly returned ([1, 2, 3, 4, 0, 6, 7, 5, 8], 1)

Testing goal_test_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8])

--- 5/5 points. Your goal_test_8p correctly True

Testing Astar_search(1, 2, 3, 4, 5, 6, 7, 0, 8],
                     actions_8p, take_action_8p,
                     lambda s: goal_test_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),
                     lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))

--- 20/20 points. Your search correctly returned ([[1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 

## Extra Credit

Add a third column for each result (from running `runExperiment`) that is the number of seconds each search required.  You may get the total run time when running a function by doing

     import time
    
     start_time = time.time()
    
     < do some python stuff >
    
     end_time = time.time()
     print('This took', end_time - start_time, 'seconds.')
